In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import datetime
from math import radians, cos, sin, asin, sqrt
import sys
import pyodbc
from sklearn.preprocessing import QuantileTransformer
import pickle
from multiprocessing import  Pool
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")


inicioapp = datetime.datetime.now()

modelo_arvore = pickle.load(open('/home/dev/IA/BACA_ARVORE/modelo_arvore_15min.sav', 'rb'))

def haversine(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 

    km = 6371* c
    return round(km, 2)


def hora_turno(x):
    if 6<x<=12:
        return 'Manha'
    elif 12<x<=17:
        return 'Tarde'
    elif 17<x<=21:
        return 'Noite'
    else:
        return 'Pernoite'
    

def dia_semana(x):
    if x == 0:
        return 'Dom'
    elif x == 1:
        return "Seg"
    elif x == 2:
        return 'Ter'
    elif x == 3:
        return 'Qua'
    elif x == 4:
        return 'Qui'
    elif x == 5:
        return 'Sex'
    else:
        return 'Sab'

def calculate_time(inicio, fim): 
    delta = fim - inicio
    return int(delta.seconds/60)#min

con = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.3.so.1.1};\
                        SERVER=cltsqlpoa15;DATABASE=apisullog;UID=apisullog;PWD=@p15u110g#')


#Realiza a leitura da query que realiza a pesquisa das SMPs que estao em monitoramento, dos clientes.
queryCliente = open("/home/dev/IA/BACA_ARVORE/Clientes-BACA.txt", "r")
queryCliente = queryCliente.read()

dfidsmp = pd.read_sql(str(queryCliente), con)
con.close()

import logging
from imp import reload
reload(logging)

dArq = inicioapp.strftime("%-d_")
mArq = inicioapp.strftime("%b")
aArq = inicioapp.strftime("_%y") # Ano com os 2 ultimos dígitos
AArq = inicioapp.strftime("%Y") # Ano com 4 digitos
nome_log = ('/home/dev/IA/BACA_ARVORE/logs/'+AArq+'/'+mArq+'/log_15m_'+'BACA_'+dArq+mArq+aArq+'.log')

path1 = "/home/dev/IA/BACA_ARVORE/logs/"+str(AArq)
try:
    os.mkdir(path1)
except OSError:
    print ("O diretório Já Existe %s failed" % path1)
else:
    print ("Diretório criado com sucesso %s " % path1)
    
path2 = "/home/dev/IA/BACA_ARVORE/logs/"+str(AArq)+"/"+str(mArq)
try:
    os.mkdir(path2)
except OSError:
    print ("O diretório Já Existe %s failed" % path2)
else:
    print ("Diretório criado com sucesso %s " % path2)

logging.basicConfig(filename=nome_log, level=logging.DEBUG,
                    format='[%(levelname)s]: [%(asctime)s] [%(message)s]', datefmt='%d-%m-%y %H:%M:%S')

smps_consulta = dfidsmp['IdSMP']
logging.info(str(len(smps_consulta)) + " SMPs em monitoramento foram coletadas com sucesso.")

smps_consulta = dfidsmp['IdSMP']
logging.info(str(len(smps_consulta)) + " SMPs em monitoramento foram coletadas com sucesso.")

def for_15_min(smps_consulta):

    smps_consulta = smps_consulta.tolist()
    contador = 0
    con = conexao = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.3.so.1.1};\
                        SERVER=cltsqlpoa15;DATABASE=apisullog;UID=apisullog;PWD=@p15u110g#')
    
    while True:   
        Numeroidsmp = smps_consulta[contador]
        #print(str(Numeroidsmp) +" Contador="+str(contador)+" QuantidadeSMPPorThread="+ str(len(smps_consulta)))
        if 1000000<= Numeroidsmp:
    
            inifor = datetime.datetime.now()
    
            queryMultisat = open('/home/dev/IA/BACA_ARVORE/Multisat.txt', 'r') 
            queryMultisat = queryMultisat.read() + str(Numeroidsmp)
    
            dfSmpMultisat = pd.read_sql(str(queryMultisat), con)
            del(queryMultisat)
        
            queryEventos = open('/home/dev/IA/BACA_ARVORE/Multisat_eventos.txt', 'r') 
            queryEventos = queryEventos.read() + str(Numeroidsmp)    

            dfSmpEventos = pd.read_sql(str(queryEventos), con)
            del(queryEventos)
    
            queryPosicoes = open('/home/dev/IA/BACA_ARVORE/Query_Posicoes_Smp.txt', 'r')
            queryPosicoes = queryPosicoes.read() + str(Numeroidsmp)
            
            dfSmpPosicao = pd.read_sql(str(queryPosicoes), con)
            del(queryPosicoes)
            
            if  dfSmpPosicao.empty == True:
                if contador < (len(smps_consulta) - 1 ):
                    contador = contador + 1
                    logging.info("IdSMP "+str(Numeroidsmp)+" nao possui posicao")
                    continue
                else:
                    smps_consulta = [0]
                    contador = 0
                    logging.info("IdSMP "+str(Numeroidsmp)+" nao possui posicao") 
                    continue

            querySubStatus = open('/home/dev/IA/BACA_ARVORE/Query_Sub_Status_Smp.txt')
            querySubStatus = querySubStatus.read() + str(Numeroidsmp)
                
            dfSmpStatusSub = pd.read_sql(str(querySubStatus), con)
            del(querySubStatus)
            
            logging.info("Dados de: "+str(Numeroidsmp)+" coletados, conexao com o banco encerrada com sucesso.")
                
            dfSmpComSinistroLocal = pd.read_csv("/home/dev/IA/BACA_ARVORE/locais_sinistros.csv", encoding="utf-8-sig", sep=";")
            
            if dfSmpMultisat['DtFimSmp'].isnull()[0] == True:
                dfSmpMultisat['DtFimSmp'] = datetime.datetime.now().strftime('%d/%m/%Y %H:%M')
                    
            if dfSmpMultisat['Valor'][0] is None:
                dfSmpMultisat['Valor'] = float(10000)
                
            dfSmpComSinistroLocal['Data'] = pd.to_datetime(dfSmpComSinistroLocal['Data'], dayfirst=True, format='%d/%m/%Y %H:%M')
            dfSmpMultisat['DtMonitoramentoSMP'] = pd.to_datetime(dfSmpMultisat['DtMonitoramentoSMP'],dayfirst=True)
            dfSmpMultisat.drop_duplicates(subset=['IdSmp'],keep='first', inplace=True)
        
            dfSmpEventos["DataInclusao"] = pd.to_datetime(dfSmpEventos["DataInclusao"])
            IdEventoConsulta = dfSmpEventos['IdEvento'].max()
                
            if IdEventoConsulta >= 0:
                pass
            else:
                IdEventoConsulta = 0
                
        
            dfSmpPosicao["DataGPSPosicao"] = pd.to_datetime(dfSmpPosicao["DataGPSPosicao"])
        
            dfSmpPosicao["Lat"] = pd.to_numeric(dfSmpPosicao["Lat"])
            dfSmpPosicao["Long"] = pd.to_numeric(dfSmpPosicao["Long"])
        
            dfSmpStatusSub["DataInclusao"] = pd.to_datetime(dfSmpStatusSub["DataInclusao"])
    
            dfSmpMultisat.dropna(subset = ["Tecnologia","DtMonitoramentoSMP"], inplace=True)
            dfSmpMultisat = dfSmpMultisat.sort_values(by=['DtMonitoramentoSMP'], ascending=False)
    
            dtPrincipal = pd.DataFrame(columns = [
            'IdSmp','Data','SubStatus','ParadoForaReferencia','DistRouboProximo',
            'UF','Cidade',
            'NaRota','KmRodado',
            'FreqSinalPrinc','MSinalPrin',
            'FreqParadRef','MParadRef',
            'FreqPernoite','MPernoite',
            'FreqParadNInf','MParadNInf',
            'FreqPortMotAb','FreqPortTraAb','FreqAlertPiscSet','FreqParadManu'   
            ])
        
            dtAlvo1 = dfSmpPosicao.iloc[0]['DataGPSPosicao']
            dtAlvo2 = dfSmpPosicao.iloc[-1]['DataGPSPosicao']
            lat = dfSmpPosicao["Lat"][-1:].values[0]
            long = dfSmpPosicao["Long"][-1:].values[0]
            distancia = 0
            subStatus = ''
            FreqSinalPrinc = 0.0
            MSinalPrin = 0.0
            FreqParadRef = 0.0
            MParadRef = 0.0
            FreqPernoite = 0.0
            MPernoite = 0.0
            FreqParadNInf = 0.0
            MParadNInf = 0.0
            FreqPortMotAb = 0.0
            FreqPortTraAb = 0.0
            FreqAlertPiscSet = 0.0
            FreqParadManu = 0.0
            ParadoForaReferencia = 0
            distanciaTemp = 0
            distanciaLocalRoubo = 0.0
        
            dfSmpEventos = dfSmpEventos.sort_values(by=['DataInclusao'], ascending=True)
            dfSmpPosicao = dfSmpPosicao.sort_values(by=['DataGPSPosicao'], ascending=True)
            dfSmpStatusSub = dfSmpStatusSub.sort_values(by=['DataInclusao'], ascending=True)
        
            distanciaTemp = distancia + haversine(long, lat, dfSmpPosicao.iloc[-1]['Long'], dfSmpPosicao.iloc[-1]['Lat'])
            localValidos = dfSmpComSinistroLocal.copy()
            distTemp = sys.maxsize
            for i, lcl in localValidos.iterrows():
                distTemp2 = haversine(long, lat, lcl['Long'], lcl['Lat'])
                if distTemp2 <distTemp:
                    distTemp = distTemp2

            distanciaLocalRoubo = distTemp
        
            if dfSmpStatusSub.empty == False:    
                subStatus = dfSmpStatusSub.iloc[-1]['Sigla']
        
            if dfSmpEventos.empty == False:    
                dfFreqSinalPrinc = dfSmpEventos[(dfSmpEventos["IdTipoEvento"].isin([257,272]))]
                FreqSinalPrinc = FreqSinalPrinc + len(dfFreqSinalPrinc)
                if FreqSinalPrinc > 0:
                    dtdiff = datetime.timedelta(microseconds=-1)
                    par = False
                    for index, row in dfSmpEventos[(dfSmpEventos["IdTipoEvento"].isin([275,272,257,345]))].iterrows():
                        if row['IdTipoEvento'] == 272 or row['IdTipoEvento'] == 257 or row['IdTipoEvento'] == 345:
                            dtInicio = row['DataInclusao']
                            par = True
                        else:
                            if par:
                                dtdiff = dtdiff + row['DataInclusao'] - dtInicio                                
                                par = False

                    MediaSinalPrincipal = dtdiff / FreqSinalPrinc
                    MSinalPrin = ((MediaSinalPrincipal * 48 )/ datetime.timedelta(days=1)) / 2
                else:
                    MSinalPrin = 0
                del(dfFreqSinalPrinc)
        
                dfFreqParadRef = dfSmpEventos[(dfSmpEventos["IdTipoEvento"].isin([42]))]
                FreqParadRef = FreqParadRef + len(dfFreqParadRef)
    
                if FreqParadRef > 0:
                    dtdiff = datetime.timedelta(microseconds=-1)
                    par = False
                    for index, row in dfSmpEventos[(dfSmpEventos["IdTipoEvento"].isin([42,58]))].iterrows():
                        if row['IdTipoEvento'] == 42:
                            dtInicio = row['DataInclusao']
                            par = True
                        else:
                            if par:
                                dtdiff = dtdiff + row['DataInclusao'] - dtInicio                                
                                par = False

                    MediaRefeicao = dtdiff / FreqParadRef
                    MParadRef = ((MediaRefeicao * 48 )/ datetime.timedelta(days=1)) / 2
                else:
                    MParadRef = 0.0
                del(dfFreqParadRef)
        
                dfFreqPernoite = dfSmpEventos[(dfSmpEventos["IdTipoEvento"].isin([41]))]
                FreqPernoite = FreqPernoite + len(dfFreqPernoite)
        
                if FreqPernoite > 0:
                    dtdiff = datetime.timedelta(microseconds=-1)
                    par = False
                    for index, row in dfSmpEventos[(dfSmpEventos["IdTipoEvento"].isin([41,141]))].iterrows():
                        if row['IdTipoEvento'] == 41:
                            dtInicio = row['DataInclusao']
                            par = True
                        else:
                            if par:
                                dtdiff = dtdiff + row['DataInclusao'] - dtInicio                                
                                par = False

                    MediaPernoite = dtdiff / FreqPernoite
                    MPernoite = ((MediaPernoite * 48 )/ datetime.timedelta(days=1)) / 2
                else:
                    MPernoite = 0.0
                    del(dfFreqPernoite)
        
                dfFreqParadNInf = dfSmpEventos[(dfSmpEventos["IdTipoEvento"].isin([154]))]
                FreqParadNInf = FreqParadNInf + len(dfFreqParadNInf)
        
                if len(dfSmpPosicao['DistanciaPontoGlobalReferencia']) >= 2:
                    if ((dfSmpPosicao['DistanciaPontoGlobalReferencia'][:-1][0] is None) and (list(dfSmpPosicao['Parado'][-1:])[0] == 1)):
                        ParadoForaReferencia = 1
                    elif ((dfSmpPosicao['DistanciaPontoGlobalReferencia'][:-1][0] is None) and (list(dfSmpPosicao['Parado'][-1:])[0] == 0)):
                        ParadoForaReferencia = 0
                    elif (len(dfSmpEventos) > 0 and dfSmpPosicao.iloc[len(dfSmpPosicao)-1]['DistanciaPontoGlobalReferencia'] >= 5000):
                        ParadoForaReferencia = 1
                    else:
                        ParadoForaReferencia = 0
                else:
                    if ((dfSmpPosicao['DistanciaPontoGlobalReferencia'][0] is None) and (dfSmpPosicao['Parado'] == 1)):
                        ParadoForaReferencia = 1
                    elif ((dfSmpPosicao['DistanciaPontoGlobalReferencia'][0] is None) and (dfSmpPosicao['Parado'] == 0)):
                        ParadoForaReferencia = 0
                    elif (len(dfSmpEventos) > 0 and dfSmpPosicao.iloc[len(dfSmpPosicao)-1]['DistanciaPontoGlobalReferencia'] >= 5000):
                        ParadoForaReferencia = 1
                    else:
                        ParadoForaReferencia = 0
    
                if FreqParadNInf > 0:
    
                    MParadNInf = (datetime.timedelta(minutes=3)*len(dfSmpEventos[(dfSmpEventos["IdTipoEvento"].isin([364]))]))
                    MParadNInf = (MParadNInf + (datetime.timedelta(minutes=10)*FreqParadNInf))/FreqParadNInf                        
                    MParadNInf = ((MParadNInf * 48 )/ datetime.timedelta(days=1)) / 2                                             
                else:
                    MParadNInf = 0.0
    
            dtPrincipal = dtPrincipal.append({'IdSmp': dfSmpMultisat['IdSmp'].values[0],'Data':dfSmpPosicao.iloc[len(dfSmpPosicao)-1]['DataGPSPosicao'],
                'SubStatus': subStatus, 'ParadoForaReferencia':ParadoForaReferencia,
                                        'DistRouboProximo' : distanciaLocalRoubo,
                'UF':dfSmpPosicao.iloc[len(dfSmpPosicao)-1]['Estado'], 'Cidade':dfSmpPosicao.iloc[len(dfSmpPosicao)-1]['Cidade'],
                'NaRota':dfSmpPosicao.iloc[len(dfSmpPosicao)-1]['NaRota'],'KmRodado': distanciaTemp, 
                                        'FreqSinalPrinc':FreqSinalPrinc, 
                                        'MSinalPrin':MSinalPrin,
                                        'FreqParadRef':FreqParadRef,
                                        'MParadRef': MParadRef,
                                        'FreqPernoite':FreqPernoite,
                                        'MPernoite': MPernoite,
                                        'FreqParadNInf':FreqParadNInf,
                                        'MParadNInf': MParadNInf,
                'FreqPortMotAb':FreqPortMotAb,'FreqPortTraAb': FreqPortTraAb,'FreqAlertPiscSet':FreqAlertPiscSet,
                'FreqParadManu': FreqParadManu
            }, ignore_index=True)    
    
    
            dtPrincipal["Semana"] = dtPrincipal['Data'].dt.weekday.apply(dia_semana)
            dtPrincipal["Turno"] = dtPrincipal['Data'].dt.hour.apply(hora_turno)
            dtPrincipal["Dia"] = dtPrincipal['Data'].dt.day
            dtPrincipal["Mes"] = dtPrincipal['Data'].dt.month
    
            dtPrincipal['IdSmp'] = pd.to_numeric(dtPrincipal['IdSmp'])
            dt = dtPrincipal.copy()
            dfCompleto = pd.merge(dt, dfSmpMultisat.drop(columns=['NumeroSMP']), on='IdSmp', how='left')
    
            dfCompleto['Cidade'] = dfCompleto['Cidade'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            dfCompleto['Cidade'] = dfCompleto['Cidade'].str.upper() 
            dfCompleto['UF'] = dfCompleto['UF'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            dfCompleto['UF'] = dfCompleto['UF'].str.upper() 
            dfCompleto['Mercadoria'] = dfCompleto['Mercadoria'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            dfCompleto['Mercadoria'] = dfCompleto['Mercadoria'].str.upper() 
    
            dfCompleto.loc[dfCompleto['SubStatus'] == '', 'SubStatus'] = 'PAR'
    
    
            tecnologias = ['AUTOTRAC', 'OMNILINK', 'ONIXSAT','OUTRAS', 'SASCAR', 'SIGHRA']
    
            # valida tipos de tecnologias válidas - caso identifique alguma diferente classifica como OUTRAS
            dfCompleto.loc[dfCompleto['Tecnologia'].isin(tecnologias)==False, 'Tecnologia'] = 'OUTRAS'
    
    
            # transforma linhas em colunas por tipo de tecnologia
            dfCompleto = pd.get_dummies(dfCompleto, prefix=['t_'], columns=['Tecnologia'])
    
            # criar colunas que estejam faltando
            tecnologias = ['t__{0}'.format(t) for t in tecnologias]
    
            for t in tecnologias:
                if t not in dfCompleto.columns:
                    dfCompleto[t] = 0
    
            dfCompleto['IdIndex'] = dfCompleto.index
            dfMolde = dfCompleto[:]['IdIndex'].to_frame()
            dfMolde = dfMolde.set_index('IdIndex')       
    
    
            col_tur = ['Turno_Manha','Turno_Noite','Turno_Pernoite','Turno_Tarde']
            for tur in col_tur: 
                dfMolde[tur] = 0
    
            col_sem = ['Semana_Dom','Semana_Qua','Semana_Qui','Semana_Sab','Semana_Seg','Semana_Sex','Semana_Ter']
            for sem in col_sem: 
                dfMolde[sem] = 0
    
            col_ope = ['Tipo Operacao_Hibrida','Tipo Operacao_Rodoviaria','Tipo Operacao_Urbana']
            for ope in col_ope: 
                dfMolde[ope] = 0
    
            col_uf = ['UF_ACRE', 'UF_ALAGOAS', 'UF_AMAPA', 'UF_AMAZONAS', 'UF_ARTIGAS', 'UF_BAHIA', 'UF_BUENOS AIRES', 'UF_CEARA',
            'UF_CHACO', 'UF_CORDOBA', 'UF_CORRIENTES', 'UF_DISTRITO FEDERAL', 'UF_ENTRE RIOS', 'UF_ESPIRITO SANTO', 'UF_FLORES',
            'UF_GOIAS', "UF_LIBERTADOR GENERAL BERNARDO O'HIGGINS", 'UF_MARANHAO', 'UF_MATO GROSSO', 'UF_MATO GROSSO DO SUL', 'UF_MAULE',
            'UF_MENDOZA', 'UF_MINAS GERAIS', 'UF_MONTEVIDEO', 'UF_PARA', 'UF_PARAIBA', 'UF_PARANA', 'UF_PAYSANDU', 'UF_PERNAMBUCO',
            'UF_PIAUI', 'UF_REGION METROPOLITANA DE SANTIAGO', 'UF_RIO DE JANEIRO', 'UF_RIO GRANDE DO NORTE', 'UF_RIO GRANDE DO SUL',
            'UF_RIO NEGRO', 'UF_RONDONIA', 'UF_RORAIMA', 'UF_SALTA', 'UF_SALTO', 'UF_SAN JOSE', 'UF_SAN LUIS', 'UF_SANTA CATARINA',
            'UF_SANTA FE', 'UF_SANTIAGO DEL ESTERO', 'UF_SAO PAULO', 'UF_SERGIPE', 'UF_SORIANO', 'UF_TOCANTINS', 'UF_TUCUMAN', 'UF_VALPARAISO']
            for uf in col_uf: 
                dfMolde[uf] = 0
    
            col_substa = ['SubStatus_A','SubStatus_AIV','SubStatus_AVL','SubStatus_FIM','SubStatus_LOG','SubStatus_PAR','SubStatus_PER']
            for substa in col_substa: 
                dfMolde[substa] = 0
    
            for index, row in dfCompleto.iterrows():
                if (row['Turno'] == "Pernoite"):
                    dfMolde.at[row['IdIndex'], 'Turno_Pernoite'] = 1
                elif (row['Turno'] == "Manha"):
                    dfMolde.at[row['IdIndex'], 'Turno_Manha'] = 1
                elif (row['Turno'] == "Noite"):
                    dfMolde.at[row['IdIndex'], 'Turno_Noite'] = 1
                elif (row['Turno'] == "Tarde"):
                    dfMolde.at[row['IdIndex'], 'Turno_Tarde'] = 1
                elif (row['Semana'] == "Qui"):
                    dfMolde.at[row['IdIndex'], 'Semana_Qui'] = 1
                elif (row['Semana'] == "Dom"):
                    dfMolde.at[row['IdIndex'], 'Semana_Dom'] = 1
                elif (row['Semana'] == "Qua"):
                    dfMolde.at[row['IdIndex'], 'Semana_Qua'] = 1
                elif (row['Semana'] == "Sex"):
                    dfMolde.at[row['IdIndex'], 'Semana_Sex'] = 1
                elif (row['Semana'] == "Ter"):
                    dfMolde.at[row['IdIndex'], 'Semana_Ter'] = 1
                elif (row['Semana'] == "Seg"):
                    dfMolde.at[row['IdIndex'], 'Semana_Seg'] = 1
                elif (row['Semana'] == "Sab"):
                    dfMolde.at[row['IdIndex'], 'Semana_Sab'] = 1
                elif (row['SubStatus'] == "A"):
                    dfMolde.at[row['IdIndex'], 'SubStatus_A'] = 1
                elif (row['SubStatus'] == "AIV"):
                    dfMolde.at[row['IdIndex'], 'SubStatus_AIV'] = 1
                elif (row['SubStatus'] == "AVL"):
                    dfMolde.at[row['IdIndex'], 'SubStatus_AVL'] = 1
                elif (row['SubStatus'] == "FIM"):
                    dfMolde.at[row['IdIndex'], 'SubStatus_FIM'] = 1
                elif (row['SubStatus'] == "LOG"):
                    dfMolde.at[row['IdIndex'], 'SubStatus_LOG'] = 1
                elif (row['SubStatus'] == "PAR"):
                    dfMolde.at[row['IdIndex'], 'SubStatus_PAR'] = 1
                elif (row['SubStatus'] == "PER"):
                    dfMolde.at[row['IdIndex'], 'SubStatus_PER'] = 1
                elif (row['Tipo Operacao'] == "Híbrida"):
                    dfMolde.at[row['IdIndex'], 'Tipo Operacao_Hibrida'] = 1
                elif (row['Tipo Operacao'] == "Rodoviária"):
                    dfMolde.at[row['IdIndex'], 'Tipo Operacao_Rodoviaria'] = 1
                elif (row['Tipo Operacao'] == "Urbana"):
                    dfMolde.at[row['IdIndex'], 'Tipo Operacao_Urbana'] = 1
                elif (row['UF'] == "ACRE"):
                    dfMolde.at[row['IdIndex'], 'UF_ACRE'] = 1
                elif (row['UF'] ==  'ALAGOAS'):
                    dfMolde.at[row['IdIndex'], 'UF_ALAGOAS'] = 1
                elif (row['UF'] ==  "AMAPA"):
                    dfMolde.at[row['IdIndex'], 'UF_AMAPA'] = 1
                elif (row['UF'] == "AMAZONAS"):
                    dfMolde.at[row['IdIndex'], 'UF_AMAZONAS'] = 1
                elif (row['UF'] == "ARTIGAS"):
                    dfMolde.at[row['IdIndex'], 'UF_ARTIGAS'] = 1
                elif (row['UF'] == "BAHIA"):
                    dfMolde.at[row['IdIndex'], 'UF_BAHIA'] = 1
                elif (row['UF'] == "BUENOS AIRES"):
                    dfMolde.at[row['IdIndex'], 'UF_BUENOS AIRES'] = 1
                elif (row['UF'] == "CEARA"):
                    dfMolde.at[row['IdIndex'], 'UF_CEARA'] = 1
                elif (row['UF'] == "CHACO"):
                    dfMolde.at[row['IdIndex'], 'UF_CHACO'] = 1
                elif (row['UF'] == "CORDOBA"):
                    dfMolde.at[row['IdIndex'], 'UF_CORDOBA'] = 1
                elif (row['UF'] == "CORRIENTES"):
                    dfMolde.at[row['IdIndex'], 'UF_CORRIENTES'] = 1
                elif (row['UF'] == "DISTRITO FEDERAL"):
                    dfMolde.at[row['IdIndex'], 'UF_DISTRITO FEDERAL'] = 1
                elif (row['UF'] == "ENTRE RIOS"):
                    dfMolde.at[row['IdIndex'], 'UF_ENTRE RIOS'] = 1
                elif (row['UF'] == "ESPIRITO SANTO"):
                    dfMolde.at[row['IdIndex'], 'UF_ESPIRITO SANTO'] = 1
                elif (row['UF'] == "FLORES"):
                    dfMolde.at[row['IdIndex'], 'UF_FLORES'] = 1
                elif (row['UF'] == "GOIAS"):
                    dfMolde.at[row['IdIndex'], 'UF_GOIAS'] = 1
                elif (row['UF'] == "LIBERTADOR GENERAL BERNARDO OHIGGINS"):
                    dfMolde.at[row['IdIndex'], 'UF_LIBERTADOR GENERAL BERNARDO OHIGGINS'] = 1
                elif (row['UF'] == "MARANHAO"):
                    dfMolde.at[row['IdIndex'], 'UF_MARANHAO'] = 1
                elif (row['UF'] == "MATO GROSSO"):
                    dfMolde.at[row['IdIndex'], 'UF_MATO GROSSO'] = 1
                elif (row['UF'] == "MATO GROSSO DO SUL"):
                    dfMolde.at[row['IdIndex'], 'UF_MATO GROSSO DO SUL'] = 1
                elif (row['UF'] == "MAULE"):
                    dfMolde.at[row['IdIndex'], 'UF_MAULE'] = 1
                elif (row['UF'] == "MENDOZA"):
                    dfMolde.at[row['IdIndex'], 'UF_MENDOZA'] = 1
                elif (row['UF'] == "MINAS GERAIS"):
                    dfMolde.at[row['IdIndex'], 'UF_MINAS GERAIS'] = 1
                elif (row['UF'] == "MONTEVIDEO"):
                    dfMolde.at[row['IdIndex'], 'UF_MONTEVIDEO'] = 1
                elif (row['UF'] == "PARA"):
                    dfMolde.at[row['IdIndex'], 'UF_PARA'] = 1
                elif (row['UF'] == "PARAIBA"):
                    dfMolde.at[row['IdIndex'], 'UF_PARAIBA'] = 1
                elif (row['UF'] == "PARANA"):
                    dfMolde.at[row['IdIndex'], 'UF_PARANA'] = 1
                elif (row['UF'] == "PAYSANDU"):
                    dfMolde.at[row['IdIndex'], 'UF_PAYSANDU'] = 1
                elif (row['UF'] == "PERNAMBUCO"):
                    dfMolde.at[row['IdIndex'], 'UF_PERNAMBUCO'] = 1
                elif (row['UF'] == "PIAUI"):
                    dfMolde.at[row['IdIndex'], 'UF_PIAUI'] = 1
                elif (row['UF'] == "REGION METROPOLITANA DE SANTIAGO"):
                    dfMolde.at[row['IdIndex'], 'UF_REGION METROPOLITANA DE SANTIAGO'] = 1
                elif (row['UF'] == "RIO DE JANEIRO"):
                    dfMolde.at[row['IdIndex'], 'UF_RIO DE JANEIRO'] = 1
                elif (row['UF'] == "RIO GRANDE DO NORTE"):
                    dfMolde.at[row['IdIndex'], 'UF_RIO GRANDE DO NORTE'] = 1
                elif (row['UF'] == "RIO GRANDE DO SUL"):
                    dfMolde.at[row['IdIndex'], 'UF_RIO GRANDE DO SUL'] = 1
                elif (row['UF'] == "RIO NEGRO"):
                    dfMolde.at[row['IdIndex'], 'UF_RIO NEGRO'] = 1
                elif (row['UF'] == "RONDONIA"):
                    dfMolde.at[row['IdIndex'], 'UF_RONDONIA'] = 1
                elif (row['UF'] == "RORAIMA"):
                    dfMolde.at[row['IdIndex'], 'UF_RORAIMA'] = 1
                elif (row['UF'] == "SALTA"):
                    dfMolde.at[row['IdIndex'], 'UF_SALTA'] = 1
                elif (row['UF'] == "SALTO"):
                    dfMolde.at[row['IdIndex'], 'UF_SALTO'] = 1
                elif (row['UF'] == "SAN JOSE"):
                    dfMolde.at[row['IdIndex'], 'UF_SAN JOSE'] = 1
                elif (row['UF'] == "SAN LUIS"):
                    dfMolde.at[row['IdIndex'], 'UF_SAN LUIS'] = 1
                elif (row['UF'] == "SANTA CATARINA"):
                    dfMolde.at[row['IdIndex'], 'UF_SANTA CATARINA'] = 1
                elif (row['UF'] == "SANTA FE"):
                    dfMolde.at[row['IdIndex'], 'UF_SANTA FE'] = 1
                elif (row['UF'] == "SANTIAGO DEL ESTERO"):
                    dfMolde.at[row['IdIndex'], 'UF_SANTIAGO DEL ESTERO'] = 1
                elif (row['UF'] == "SAO PAULO"):
                    dfMolde.at[row['IdIndex'], 'UF_SAO PAULO'] = 1
                elif (row['UF'] == "SERGIPE"):
                    dfMolde.at[row['IdIndex'], 'UF_SERGIPE'] = 1
                elif (row['UF'] == "SORIANO"):
                    dfMolde.at[row['IdIndex'], 'UF_SORIANO'] = 1
                elif (row['UF'] == "TOCANTINS"):
                    dfMolde.at[row['IdIndex'], 'UF_TOCANTINS'] = 1
                elif (row['UF'] == "TUCUMAN"):
                    dfMolde.at[row['IdIndex'], 'UF_TUCUMAN'] = 1
                elif (row['UF'] == "VALPARAISO"):
                    dfMolde.at[row['IdIndex'], 'UF_VALPARAISO'] = 1
    
    
            dfCompleto = pd.merge(dfMolde, dfCompleto, on='IdIndex', how='left')
            del(col_tur, tur, col_sem, sem, col_ope, ope, col_substa, substa, col_uf, uf)    
    
            del(dt)
            df = dfCompleto.copy()
            del(dfCompleto, dfSmpPosicao, dfMolde, dfSmpEventos, dfSmpMultisat, dfSmpStatusSub)
    
            df['NaRota'] = df['NaRota'].astype(int) #Transforma o bool em int
    
            materiais_de_construcao = ['MADERITE',
            'CAL',
            'PEDRAS',
            'CALCARIO E GESSO',
            'CIMENTO',
            'TIGRE S.A',
            'ARTIGOS CONSTRUCAO',
            'AREIA',
            'TINTAS IMOBILIARIAS',
            'AGLOMERADO',
            'FERRAMENTAS']
    
            df.loc[df['Mercadoria'].isin(materiais_de_construcao)==True, 'Mercadoria'] = 'MATERIAIS DE CONSTRUCAO'
    
            papeis = ['CELULOSE',
            'PAPEL',
            'PAPEL INDUSTRIALIZADO',
            'CAIXAS DE PAPELAO',
            'MATERIAL GRAFICO',
            'MATERIAL GRAFICO EXPRESSO',
            'ADESIVOS',
            'PAPIER FABRIK WATTENS',
            'METSA']
    
            df.loc[df['Mercadoria'].isin(papeis)==True, 'Mercadoria'] = 'PAPEIS'
    
            cigarros = ['PAPEL DE CIGARRO',
            'CIGARRO DE INIBICAO',
            'TABACO',
            'TABACO INFESTADO',
            'FUMO',
            'CARREGAMENTO CONTAINER A TABACO INFESTADO',
            'CARREGAMENTO CONTAINER B TABACO INFESTADO',
            'BAT CHILE',
            'PT ARGHA',
            'IMPORTACAO PT ARGHA KARIA',
            'IMPORTACAO ARGHA KARYA',
            'ARGHA KARYA']
    
            df.loc[df['Mercadoria'].isin(cigarros)==True, 'Mercadoria'] = 'CIGARROS'
    
            moveis = ['MESAS E CADEIRAS']
    
            df.loc[df['Mercadoria'].isin(moveis)==True, 'Mercadoria'] = 'MOVEIS'
    
            escolar = ['BIC',
            'CADERNOS',
            'CADERNOS E AGENDAS',
            'MATERIAL DE ESCRITORIO',
            'ARTIGOS ESCOLARES']
    
            df.loc[df['Mercadoria'].isin(escolar)==True, 'Mercadoria'] = 'ESCOLAR'
    
            armamento = ['MATERIAL BELICO']
    
            df.loc[df['Mercadoria'].isin(armamento)==True, 'Mercadoria'] = 'ARMAMENTO'
    
            material_eletronico = ['FIOS E CABOS',
            'EQUIPAMENTOS ELETRICOS',
            'PAINEIS ELETRICOS',
            'POSTE',
            'CABOS DE FIBRA OTICA',
            'TRANSFORMADOR',
            'GERADORES',
            'CABOS ELETRICOS']
    
            df.loc[df['Mercadoria'].isin(material_eletronico)==True, 'Mercadoria'] = 'MATERIAL ELETRICO'
    
            eletronicos = ['CAMERAS FOTOGRAFICAS',
            'COMPONENTES ELETRONICOS',
            'CD E DVD',
            'CELULAR',
            'BATERIAS P/ CELULAR',
            'MAQUINAS FOTOGRAFICAS',
            'CARTOES TELEFONICOS',
            'ARTIGOS FOTOGRAFICOS',
            'PHILIPS',
            'HITACHI']
    
            df.loc[df['Mercadoria'].isin(eletronicos)==True, 'Mercadoria'] = 'ELETRONICOS'
    
            diversos = ['DIVERSOS',
            'CARGA FRACIONADA',
            'CARGA MISTA',
            'DIVERSOS PRODUTOS',
            'ESPECIFICOS',
            'SEM ESPECIFICACAO',
            'MERCADORIAS ESPECIFICAS',
            'CARGA CONSOLIDADA',
            'CARGA DE TERCEIRO',
            'MANE',
            'KATO',
            'IMPORTACAO MATERIAL GRAFICO',
            'IMPORTACAO KATO',
            'CARREGAMENTO REVERSA PARA SANTA CRUZ DO SUL',
            'CARREGAMENTO REVERSA PARA CACHOEIRINHA',
            'CARREGAMENTO FMD',
            'CARREGAMENTO M O ARMAZEM MATERIAIS PARA DESTRUICAO',
            'VAZIO',
            'MANUTENCAO',
            'PRODUTOS DESCARTAVEIS',
            'PSD',
            'PRODUTO ACABADO' 
            ]
    
            df.loc[df['Mercadoria'].isin(diversos)==True, 'Mercadoria'] = 'DIVERSOS'
    
            medicamentos = ['UNIDOCKS(MEDICAMENTOS)',
            'MATERIAL HOSPITALAR',
            'INSUMOS PARA FABRICACAO DE MEDICAMENTOS',
            'INSULINA',
            'MEDICAMENTOS QUIMICOS VETERINARIO',
            'PRODUTOS HOSPITALARES',
            'MEDICAMENTOS HUMANOS',
            'PRODUTO FARMACEUTICO',
            'PRODUTO QUIMICO FARMACEUTICO',
            'EQUIPAMENTOS MEDICOS',
            'SORO FISIOLOGICO']
    
            df.loc[df['Mercadoria'].isin(medicamentos)==True, 'Mercadoria'] = 'MEDICAMENTOS'
    
            roupas = ['MATERIAL ESPORTIVO',
            'MALHAS',
            'COURO WET BLUE',
            'COURO IN NATURA',
            'FIO TEXTIL',
            'TECIDOS',
            'PLUMA DE ALGODAO',
            'COURO',
            'COURO CRU',
            'CONFECCOES',
            'CALCADOS',
            'ROUPAS',
            'ALGODAO',
            'ALPARGATA',
            'ARTIGOS ESPORTIVOS']
    
            df.loc[df['Mercadoria'].isin(roupas)==True, 'Mercadoria'] = 'ROUPAS'
    
            produtos_de_limpeza = ['HENKEL']
    
            df.loc[df['Mercadoria'].isin(produtos_de_limpeza)==True, 'Mercadoria'] = 'PRODUTOS DE LIMPEZA'
    
            carnes = ['CARNES IN NATURA',
            'FRANGO',
            'PESCADOS CONGELADOS',
            'CHARQUE',
            'CARNES',
            'CARNES OVINAS',
            'CARNES BOVINAS',
            'VERGALHO BOVINO',
            'VERGALHO CONGELADO',
            'SALMAO FRESCO',
            'SALMAO CONGELADO']
    
            df.loc[df['Mercadoria'].isin(carnes)==True, 'Mercadoria'] = 'CARNES'
    
            metais = ['BOBINAS DE ACO',
            'BOBINAS DE ALUMINIO',
            'ALUMINIO',
            'CHAPAS DE ACO',
            'VERGALHOES',
            'COBRE EM BARRAS',
            'COBRE EM LINGOTES',
            'COBRE EM TARUGOS',
            'COBRE EM CHAPAS',
            'COBRE EM FIOS',
            'COBRE EM SUCATAS',
            'ACO',
            'COBRE BRUTO',
            'COBRE',
            'ALUMINIO EM BARRAS',
            'ALUMINIO EM LINGOTES',
            'ALUMINIO EM TARUGOS',
            'ALUMINIO EM FIOS',
            'BOBINA',
            'TUBOS',
            'ESTANHO',
            'ESTRUTURAS METALICAS',
            'LIGA DE ALUMINIO',
            'OXIDO DE ALUMINIO',
            'PRODUTOS SIDERURGICOS',
            'MINERIOS EM GERAL',
            'NIQUEL',
            'FERRAGENS']
    
            df.loc[df['Mercadoria'].isin(metais)==True, 'Mercadoria'] = 'METAIS'
    
            auto_pecas = ['BOSCH',
            'GM(PECAS)',
            'CAMARA DE AR',
            'PNEUS',
            'PIRELLI',
            'PECAS',
            'ROLAMENTOS',
            'EMBALAGENS AUTOMOTIVAS',
            'PECAS AUTOMOTIVAS',
            'MOTORES',
            'BATERIAS',
            'OLEO MINERAL',
            'PARKSIDE',
            'MATERIAL REPROCESSADO']
    
            df.loc[df['Mercadoria'].isin(auto_pecas)==True, 'Mercadoria'] = 'AUTO-PECAS'
    
            combustivel = ['OLEO LUBRIFICANTE',
            'OLEO DIESEL',
            'BIODIESEL',
            'ALCOOL COMBUSTIVEL',
            'ALCOOL',
            'OLEO COMESTIVEL',
            'ALCOOL ETILICO ANIDRO',
            'ALCOOL ETILICO ANIDRO COMBUSTIVEL',
            'ALCOOL ETILICO HIDRATADO',
            'ALCOOL ETILICO HIDRATADO COMBUSTIVEL',
            'ALCOOL FURFURILICO',
            'DIESEL',
            'DIESEL B',
            'GASOLINA C COMUM',
            'GASOLINA COMUM',
            'GASOLINA PETROBRAS',
            'GASOLINA SUPRA - LI',
            'OLEO DIESEL/BIODIESEL',
            'OLEO DIESEL/BIODIESEL B2',
            'GASOLINA A',
            'OLEO DE MOTOR EXTRA VIDA',
            'METANOL',
            'DIACETONA ALCOOL',
            'OLEO COMBUSTIVEL A1 -',
            'ETANOL',
            'INFLAMAVEIS',
            'OCTANOL',
            'BIODISEL B100',
            'PETROLEO']
    
            df.loc[df['Mercadoria'].isin(combustivel)==True, 'Mercadoria'] = 'COMBUSTIVEL'
    
            derivados_de_leite = ['LEITE EM PO',
            'LEITE',
            'MADEIRA',
            'QUEIJO',
            'IOGURTE',
            'SORO DE LEITE',
            'LACTEOS REFRIGERADOS',
            'LACTEOS SECOS']
    
            df.loc[df['Mercadoria'].isin(derivados_de_leite)==True, 'Mercadoria'] = 'DERIVADOS DE LEITE'
    
            alimentos_em_geral = ['ALIMENTOS ACHOCOLATADOS',
            'BISCOITO',
            'NESTLE',
            'ALIMENTOS NAO PERECIVEL',
            'FRUTAS',
            'GENERO ALIMENTICIO NAO PERECIVEL',
            'VERDURAS',
            'CHOCOLATE',
            'BATATAS CONGELADAS',
            'VEGETAIS E VERDURAS',
            'FRUTAS RESFRIADAS',
            'MERCEARIA',
            'MERENDA ESCOLAR',
            'HORTIFRUTI',
            'NISSIN',
            'CONGELADOS',
            'PROTEINA',
            'ATOMATADOS',
            'ADRIA ALIMENTOS',
            'CARGILL',
            'BARRY CALEBAUT',
            'OLEO DE LINHACA',
            'CAFE EM GRAOS',
            'CAFE BENEFICIADO',
            'CAFE',
            'MATERIA PRIMA DE CACAU',
            'CACAU IN NATURA',
            'CACAU',
            'ALHO',
            'MATERIA PRIMA PARA GOMA DE MASCAR',
            'SOJA',
            'TRIGO',
            'MILHO',
            'ACUCAR',
            'ARROZ',
            'ACUCAR EM BAGS',
            'GORDURA VEGETAL']
    
            df.loc[df['Mercadoria'].isin(alimentos_em_geral)==True, 'Mercadoria'] = 'ALIMENTOS EM GERAL'
    
            higiene_pessoal = ['COLGATE PALMOLIVE',
            'UNILEVER',
            'SABAO EM PO',
            'HIGIENE E LIMPEZA',
            'GILETTE',
            'JOHNSON']
    
            df.loc[df['Mercadoria'].isin(higiene_pessoal)==True, 'Mercadoria'] = 'HIGIENE PESSOAL'
    
            computadores = ['MATERIAL DE INFORMATICA',
            'DELL',
            'INFORMATICA - PECAS - SUPRIMENTOS',
            'SERVIDORES',
            'POSITIVO']
    
            df.loc[df['Mercadoria'].isin(computadores)==True, 'Mercadoria'] = 'COMPUTADORES'
    
            eletro_eletronicos = ['ELETRODOMESTICOS',
            'ELGIN']
    
            df.loc[df['Mercadoria'].isin(eletro_eletronicos)==True, 'Mercadoria'] = 'ELETRO-ELETRONICOS'
    
            veiculos = ['VOLKSWAGEN',
            'MOTO',
            'MATERIAL HB FULLER']
    
            df.loc[df['Mercadoria'].isin(veiculos)==True, 'Mercadoria'] = 'VEICULOS'
    
            agricola = ['DEFENSIVO AGRICOLA',
            'RACAO ANIMAL',
            'FERTILIZANTES',
            'IMPLEMENTOS AGRICOLAS',
            'MAQUINAS AGRICOLAS',
            'TRATORES',
            'AGRIBUSINESS',
            'FARELO',
            'ADUBOS',
            'SEMENTE',
            'OLEO PULVERIZACAO AGRICOLA BR - CE',
            'MEDICAMENTOS ANIMAIS',
            'BAGACO DE CANA',
            'CE ROEPER GMBH',
            'MAFCO']
    
            df.loc[df['Mercadoria'].isin(agricola)==True, 'Mercadoria'] = 'AGRICOLA'
    
            artigos_domesticos = ['ART. CUTELARIA',
            'ART. BAZAR',
            'ABAJURES',
            'ARTESANATO',
            'TRAMONTINA',
            'CABO ACETATO']
    
            df.loc[df['Mercadoria'].isin(artigos_domesticos)==True, 'Mercadoria'] = 'ARTIGOS DOMESTICOS'
    
            equipamentos_pesados = ['DOSADOR',
            'PICADOR',
            'EVAPORADOR',
            'CONDENSADOR',
            'ISOTANQUES',
            'MAQUINAS INDUSTRIAIS',
            'MAQUINARIOS',
            'PONTE ROLANTE',
            'TURBINAS',
            'PLANTAS INDUSTRIAIS',
            'SILO',
            'TANQUE',
            'PALETES ESPULAS E TAMPAS',
            'ISOTANK']
    
            df.loc[df['Mercadoria'].isin(equipamentos_pesados)==True, 'Mercadoria'] = 'EQUIPAMENTOS PESADOS'
    
            embalagens = ['PALLETS',
            'GARRAFAS VAZIAS',
            'CARREGAMENTO CONTAINER 1',
            'EMBALAGENS PLASTICAS',
            'VASILHAME',
            'TANN COLOMBIANA SA',
            'PARKSIDE FLEXIBLES ASIA',
            'PACKDUQUE',
            'INNOVIA FILMS',
            'IMPORTACAO INNOVIA FILMS',
            'DESCARGA MO ARMAZEM BOMBONAS',
            'CCL',
            'CARREGAMENTO M O ARMAZEM',
            'CAIXAS C48',
            'BEUCKE']
    
            df.loc[df['Mercadoria'].isin(embalagens)==True, 'Mercadoria'] = 'EMBALAGENS'
    
            bebidas = ['CERVEJA',
            'REFRIGERANTE',
            'VINHOS',
            'BEBIDAS DESTILADAS',
            'EXTRATO DE MALTE',
            'AGUA DE COCO',
            'AGUA MINERAL',
            'SUCO DE LARANJA']
    
            df.loc[df['Mercadoria'].isin(bebidas)==True, 'Mercadoria'] = 'BEBIDAS'
    
            quimicos = ['POLIETILENO',
            'ESTIRENO',
            'SOLVENTE',
            'SOLVENTE ALIFATICO LEVE',
            'SOLVENTE MTS',
            'SOLVENTE PARA BORRACHA',
            'PELICULA DE POLIETILENO',
            'POLIESTIRENO',
            'POLICLORETO DE ALUMINIO ( COAG OR 100)',
            'PROPILENO GLICOL INDUSTRIAL',
            'TINTAS E SOLVENTES',
            'CORROSIVOS',
            'DIOXIDO DE CARBONO',
            'FERRO NIOBIO',
            'FOSFORO',
            'OLEO BPF',
            'BOVALIS (COMPOSTO DE POLIMEROS)',
            'RESINAS TERMOPLATICAS',
            'PRODUTOS QUIMICOS',
            'QUIMICA',
            'DIOXIDO DE TITANIO',
            'ACETATO DE BUTILA',
            'ACETATO DE ETIL GLICOL',
            'ACETATO DE ETILA',
            'ACETATO DE ISOPENTILA',
            'ACETATO DE VINILA',
            'ACETONA',
            'ACIDO ETIL HEXANOICO',
            'ACIDO OXALICO',
            'ACIDO SULFONICO',
            'DERIVADOS DE PETROLEO',
            'GLICERINA',
            'TOLUENO',
            'SEC BUTANOL',
            'SODA CAUSTICA 98',
            'SODA CAUSTICA PEROLADA',
            'SOLUCAO AMONIACAL',
            'CICLOHEXANO',
            'CICLOHEXANONA',
            'METILATO DE SODIO',
            'METILETILCETONA',
            'METILETILCETONA (MEC)',
            'MONOETANOLAMINA (MEA)',
            'MONOETILENOGLICOL GF',
            'MONOETILENOGLICOL-H',
            'RESINA FENOLICA',
            'POLIURETANO',
            'POLIPROPILENO',
            'NORMAL BUTANOL (NBA)',
            'NORMAL PARAFINA C-13 +HIDROG.',
            'ESTIRENO',
            'ETILA',
            'ETILBENZENO',
            'ETILGLICOL',
            'MOLIBIDENIO',
            'PENTOXIDO',
            'SOLUCAO SULFOCAUSTICA',
            'SOLVENTE AB',
            'RESINA',
            'FORMOL',
            'BUTILDIGLICOL',
            'BUTILGLICOL',
            'ISOPROPANOL',
            'QUIMICA ANASTACIO',
            'RAFINADO C6C8',
            'BUTENO 1',
            'SOLVENTE AB9',
            'ACRILONITRILA',
            'ISOBUTANOL',
            'AGUA DESMINERALIZADA',
            'XILENOS MISTOS',
            'BORRACHA NATURAL',
            'IMPORTACAO TENNANTS FINE CHEMICALS LTDA',
            'IBCS',
            'HB FULLER',
            'EXPORTACAO ESSENCIA CAE',
            'ESSENTRA',
            'ESSENTRA',
            'DAICEL',
            'CARTON DRUCK',
            'AMPLIGO IMPORTACAO']
    
            df.loc[df['Mercadoria'].isin(quimicos)==True, 'Mercadoria'] = 'QUIMICOS'
    
            df.loc[df['Conta'].isin(["BRF CTB"])==True, 'Mercadoria'] = 'CARNES'
            df.loc[df['Conta'].isin(["MOURA BATER"])==True, 'Mercadoria'] = 'AUTO-PECAS'
            df.loc[df['Conta'].isin(["QUIMICA AMPA"])==True, 'Mercadoria'] = 'PRODUTOS DE LIMPEZA'
            df.loc[df['Conta'].isin(["LACTALIS"])==True, 'Mercadoria'] = 'DERIVADOS DE LEITE'
            df.loc[df['Conta'].isin(["CNH"])==True, 'Mercadoria'] = 'VEICULOS'
            df.loc[df['Conta'].isin(["MONDELEZ BRA"])==True, 'Mercadoria'] = 'ALIMENTOS EM GERAL'
            df.loc[df['Conta'].isin(["ELECTROLUX D"])==True, 'Mercadoria'] = 'ELETRO-ELETRONICOS'
            df.loc[df['Conta'].isin(["BRF JUNDIAI"])==True, 'Mercadoria'] = 'CARNES'
            df.loc[df['Conta'].isin(["CEREAIS DO N"])==True, 'Mercadoria'] = 'AGRICOLA'
    
            cidades = ['SAO PAULO',
            'RIO DE JANEIRO',
            'DUQUE DE CAXIAS',
            'CONTAGEM',
            'GUARULHOS',
            'CURITIBA',
            'PORTO ALEGRE',
            'SUMARE',
            'SALVADOR',
            'CAMPINAS',
            'JUNDIAI',
            'OSASCO',
            'GOIANIA',
            'BETIM',
            'SAO BERNARDO DO CAMPO',
            'FORTALEZA',
            'EMBU DAS ARTES',
            'CANOAS',
            'SAO JOSE DOS CAMPOS',
            'BARUERI',
            'ANAPOLIS',
            'MANAUS',
            'CABO DE SANTO AGOSTINHO',
            'NOVO HAMBURGO',
            'BELEM',
            'HORTOLANDIA',
            'APARECIDA DE GOIANIA',
            'RIBEIRAO PRETO',
            'VITORIA DE SANTO ANTAO',
            'TERESINA',
            'SANTOS',
            'JOINVILLE',
            'ITAJAI',
            'SAO JOSE DOS PINHAIS',
            'BRASILIA',
            'PARANAGUA',
            'PONTA GROSSA',
            'BELO JARDIM',
            'PAULINIA',
            'CACHOEIRINHA',
            'MARACANAU',
            'NAVEGANTES',
            'FEIRA DE SANTANA',
            'LONDRINA',
            'UBERLANDIA',
            'PALHOCA',
            'JABOATAO DOS GUARARAPES',
            'NOVA SANTA RITA',
            'VIANA',
            'LOUVEIRA',
            'CAJAMAR',
            'ITAITINGA',
            'VIDEIRA',
            'ANANINDEUA',
            'CAMPO GRANDE',
            'TEUTONIA',
            'CAMACARI',
            'CHAPECO',
            'SOROCABA',
            'BENTO GONCALVES',
            'OUTRAS']
    
            df.loc[df['Cidade'].isin(cidades)==False, 'Cidade'] = 'OUTRAS'
    
            df = pd.get_dummies(df, prefix=['c_'], columns=['Cidade'])
    
            cidades = ['c__{0}'.format(c) for c in cidades]
    
            for c in cidades:
                if c not in df.columns:
                    df[c] = 0
    
    
            mercadorias = ['BEBIDAS','EMBALAGENS','EQUIPAMENTOS PESADOS','ARTIGOS DOMESTICOS','AGRICOLA','VEICULOS',
                'ELETRO-ELETRONICOS','COMPUTADORES','HIGIENE PESSOAL','ALIMENTOS EM GERAL','DERIVADOS DE LEITE',
                'COMBUSTIVEL','AUTO-PECAS','METAIS','OUTRAS','CARNES','QUIMICOS','PRODUTOS DE LIMPEZA','ROUPAS','MEDICAMENTOS',
                'ELETRONICOS','MATERIAL ELETRICO','ARMAMENTO','ESCOLAR',
                'MOVEIS','CIGARROS','PAPEIS','DIVERSOS','MATERIAIS DE CONSTRUCAO']
    
            df.loc[df['Mercadoria'].isin(mercadorias)==False, 'Mercadoria'] = 'OUTRAS'
    
            df = pd.get_dummies(df, prefix=['m_'], columns=['Mercadoria'])
    
            mercadorias = ['m__{0}'.format(m) for m in mercadorias]
    
            for m in mercadorias:
                if m not in df.columns:
                    df[m] = 0
    
    
            
    
            df.drop(columns=['Conta', 'Dia', 'Mes', 'Data', 'DtMonitoramentoSMP', 'DtFimSmp', 'IdSmp', 'SubStatus',
                    'Semana', 'Turno', 'Tipo Operacao', 'IdIndex'], inplace=True)

            df = df[['ParadoForaReferencia','DistRouboProximo','NaRota','KmRodado','FreqSinalPrinc','MSinalPrin','FreqParadRef','MParadRef',
            'FreqPernoite','MPernoite','FreqParadNInf','MParadNInf','FreqPortMotAb','FreqPortTraAb','FreqAlertPiscSet','FreqParadManu',
            'Valor','t__AUTOTRAC','t__OMNILINK','t__ONIXSAT','t__OUTRAS','t__SASCAR','t__SIGHRA','Turno_Manha','Turno_Noite','Turno_Pernoite',
            'Turno_Tarde','Semana_Dom','Semana_Qua','Semana_Qui','Semana_Sab','Semana_Seg','Semana_Sex','Semana_Ter','Tipo Operacao_Hibrida',
            'Tipo Operacao_Rodoviaria','Tipo Operacao_Urbana','UF_ACRE','UF_ALAGOAS','UF_AMAPA','UF_AMAZONAS','UF_ARTIGAS','UF_BAHIA',
            'UF_BUENOS AIRES','UF_CEARA','UF_CHACO','UF_CORDOBA','UF_CORRIENTES','UF_DISTRITO FEDERAL','UF_ENTRE RIOS','UF_ESPIRITO SANTO',
            'UF_FLORES','UF_GOIAS',"UF_LIBERTADOR GENERAL BERNARDO O'HIGGINS",'UF_MARANHAO','UF_MATO GROSSO','UF_MATO GROSSO DO SUL',
            'UF_MAULE','UF_MENDOZA','UF_MINAS GERAIS','UF_MONTEVIDEO','UF_PARA','UF_PARAIBA','UF_PARANA','UF_PAYSANDU','UF_PERNAMBUCO',
            'UF_PIAUI','UF_REGION METROPOLITANA DE SANTIAGO','UF_RIO DE JANEIRO','UF_RIO GRANDE DO NORTE','UF_RIO GRANDE DO SUL',
            'UF_RIO NEGRO','UF_RONDONIA','UF_RORAIMA','UF_SALTA','UF_SALTO','UF_SAN JOSE','UF_SAN LUIS','UF_SANTA CATARINA','UF_SANTA FE',
            'UF_SANTIAGO DEL ESTERO','UF_SAO PAULO','UF_SERGIPE','UF_SORIANO','UF_TOCANTINS','UF_TUCUMAN','UF_VALPARAISO','SubStatus_A',
            'SubStatus_AIV','SubStatus_AVL','SubStatus_FIM','SubStatus_LOG','SubStatus_PAR','SubStatus_PER','c__ANANINDEUA','c__ANAPOLIS',
            'c__APARECIDA DE GOIANIA','c__BARUERI','c__BELEM','c__BELO JARDIM','c__BENTO GONCALVES','c__BETIM','c__BRASILIA','c__CABO DE SANTO AGOSTINHO',
            'c__CACHOEIRINHA','c__CAJAMAR','c__CAMACARI','c__CAMPINAS','c__CAMPO GRANDE','c__CANOAS','c__CHAPECO','c__CONTAGEM','c__CURITIBA',
            'c__DUQUE DE CAXIAS','c__EMBU DAS ARTES','c__FEIRA DE SANTANA','c__FORTALEZA','c__GOIANIA','c__GUARULHOS','c__HORTOLANDIA',
            'c__ITAITINGA','c__ITAJAI','c__JABOATAO DOS GUARARAPES','c__JOINVILLE','c__JUNDIAI','c__LONDRINA','c__LOUVEIRA','c__MANAUS',
            'c__MARACANAU','c__NAVEGANTES','c__NOVA SANTA RITA','c__NOVO HAMBURGO','c__OSASCO','c__OUTRAS','c__PALHOCA','c__PARANAGUA',
            'c__PAULINIA','c__PONTA GROSSA','c__PORTO ALEGRE','c__RIBEIRAO PRETO','c__RIO DE JANEIRO','c__SALVADOR','c__SANTOS',
            'c__SAO BERNARDO DO CAMPO','c__SAO JOSE DOS CAMPOS','c__SAO JOSE DOS PINHAIS','c__SAO PAULO','c__SOROCABA','c__SUMARE',
            'c__TERESINA','c__TEUTONIA','c__UBERLANDIA','c__VIANA','c__VIDEIRA','c__VITORIA DE SANTO ANTAO','m__AGRICOLA','m__ALIMENTOS EM GERAL',
            'm__AUTO-PECAS','m__BEBIDAS','m__CARNES','m__CIGARROS','m__COMBUSTIVEL','m__DERIVADOS DE LEITE','m__DIVERSOS','m__ELETRO-ELETRONICOS',
            'm__EMBALAGENS','m__HIGIENE PESSOAL','m__MEDICAMENTOS','m__METAIS','m__OUTRAS','m__PAPEIS','m__PRODUTOS DE LIMPEZA','m__QUIMICOS',
            'm__ROUPAS','m__VEICULOS','m__EQUIPAMENTOS PESADOS','m__ARTIGOS DOMESTICOS','m__COMPUTADORES','m__ELETRONICOS','m__MATERIAL ELETRICO',
            'm__ARMAMENTO','m__ESCOLAR','m__MOVEIS','m__MATERIAIS DE CONSTRUCAO']]
        
        
            Imput = np.array(df)
            df['Consulta'] = datetime.datetime.now()
            df.to_csv('/home/dev/IA/BACA_ARVORE/linha_smp/'+str(Numeroidsmp)+'.txt',header=None, index=None, sep=';', mode='a')
            del(df)
            
            probabilidade = modelo_arvore.predict_proba(Imput)[:, 1]
            probabilidade = round(probabilidade[0], 5)
    
            fim_for = datetime.datetime.now()
            logging.info("A verificacao de "+ str(Numeroidsmp)+" levou " + str(fim_for - inifor))
        
            IdEventoConsulta = str(IdEventoConsulta)
            DataConsulta = inifor.strftime('%d/%m/%Y %H:%M:%S.%f')
            DataConsulta = pd.to_datetime(DataConsulta, dayfirst=True)
                
            if contador < (len(smps_consulta) - 1 ):
                contador = contador + 1
            else:
                smps_consulta = [0]
                contador = 0
                
            try:
                cur = con.cursor()
                cur.execute("EXECUTE GravaProbabilidade ?, ?, ?, ?",
                    (Numeroidsmp, DataConsulta, probabilidade, int(IdEventoConsulta)))
                con.commit()
            finally:
                cur.close() # Fechando o cursor.
        
        else:
            if str(Numeroidsmp) == "0":
                fim_for = datetime.datetime.now()
                logging.info("Thread Finalizada com sucesso! A Thread durou: " + str(fim_for - inicioapp))
                break
        
    con.close()
    print("Fim Thread")
    
threads = 6
print("Rodando "+ str(threads)+" Threads")
smps_split = np.array_split(smps_consulta, threads)
pool = Pool(threads)
pool.map(for_15_min, smps_split)
pool.close()
pool.join()
final_processamento_Total = datetime.datetime.now()
logging.info("Programa Finalizado com sucesso! A aplicação durou: " + str(final_processamento_Total - inicioapp))
print("acabou! " + str(final_processamento_Total - inicioapp))

Using TensorFlow backend.


O diretório Já Existe /home/dev/IA/BACA_ARVORE/logs/2021 failed
O diretório Já Existe /home/dev/IA/BACA_ARVORE/logs/2021/Mar failed
Rodando 6 Threads
Fim Thread
Fim Thread
Fim Thread
Fim Thread
Fim Thread
Fim Thread
acabou! 0:11:08.509957
2021-03-04 10:45:10.807178


In [2]:
str(datetime.datetime.now())

'2021-03-04 10:45:10.831566'